<a href="https://colab.research.google.com/github/SJG0510/ML-Project/blob/main/Health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 라이브러리 불러오기
import logging
import pandas as pd

# matplotlib 경고 제거
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

# GitHub에서 엑셀 파일 다운로드
!wget "https://raw.githubusercontent.com/SJG0510/SJG-ML-Project/data/2024년_3-1.부록(시도별)-(1)건강행태.xlsx" -O health_sido.xlsx
!wget "https://raw.githubusercontent.com/SJG0510/SJG-ML-Project/data/2024년_3-2.부록(시군구별)-(1)건강행태.xlsx" -O health_sigungu.xlsx
!wget "https://raw.githubusercontent.com/SJG0510/ML-Project/data/2024년_3-1.부록(시도별)-당뇨 및 고혈압.xlsx" -O health_sido_disease.xlsx
!wget "https://raw.githubusercontent.com/SJG0510/ML-Project/data/2024년_3-2.부록(시군구별)-당뇨 및 고혈압.xlsx" -O health_sigungu_disease.xlsx
!wget "https://raw.githubusercontent.com/SJG0510/ML-Project/data/알코올성%20간질환%2010세구간별현황_2019_2023.xlsx" -O alcohol_liver_disease.xlsx
!wget "https://raw.githubusercontent.com/SJG0510/ML-Project/data/지역별%20우울증%20진료현황(2019년~2023년).xlsx" -O depression_by_region.xlsx
!wget "https://raw.githubusercontent.com/SJG0510/ML-Project/data/폐암%20발생자수.xlsx" -O lung_cancer_cases.xlsx

# 엑셀 파일 로드
xls_province = pd.ExcelFile("health_sido.xlsx")      # 시도별 데이터
xls_district = pd.ExcelFile("health_sigungu.xlsx")   # 시군구별 데이터
xls_province_disease = pd.ExcelFile("health_sido_disease.xlsx")    # 시도별 당뇨/고혈압
xls_district_disease = pd.ExcelFile("health_sigungu_disease.xlsx") # 시군구별 당뇨/고혈압
df_alcohol_liver = pd.read_excel("alcohol_liver_disease.xlsx")         # 연령대별 알코올성 간질환 현황 (10세 구간, 2019~2023년)
df_depression_region = pd.read_excel("depression_by_region.xlsx")      # 지역별 우울증 진료 현황 (2019~2023년)
df_lung_cancer = pd.read_excel("lung_cancer_cases.xlsx")               # 폐암 발생자 수

--2025-05-26 10:16:43--  https://raw.githubusercontent.com/SJG0510/SJG-ML-Project/data/2024%EB%85%84_3-1.%EB%B6%80%EB%A1%9D(%EC%8B%9C%EB%8F%84%EB%B3%84)-(1)%EA%B1%B4%EA%B0%95%ED%96%89%ED%83%9C.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 723727 (707K) [application/octet-stream]
Saving to: ‘health_sido.xlsx’

health_sido.xlsx    100%[===================>] 706.76K  --.-KB/s    in 0.04s   

2025-05-26 10:16:43 (17.7 MB/s) - ‘health_sido.xlsx’ saved [723727/723727]

--2025-05-26 10:16:43--  https://raw.githubusercontent.com/SJG0510/SJG-ML-Project/data/2024%EB%85%84_3-2.%EB%B6%80%EB%A1%9D(%EC%8B%9C%EA%B5%B0%EA%B5%AC%EB%B3%84)-(1)%EA%B1%B4%EA%B0%95%ED%96%89%ED%83%9C.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 18

In [ ]:
# 확인할 시트 목록
target_sheets = ['1.현재흡연', '7.월간음주', '13.비만율', '17.스트레스', '18.우울감']

# 시도별 엑셀 객체
xls = pd.ExcelFile("health_sido.xlsx")

# 각 시트에서 C12:W199 범위 일부 확인
for sheet in target_sheets:
    print(f"\n📄 시트: {sheet}")
    df = xls.parse(sheet, skiprows=11, usecols="C:W", nrows=11)
    display(df)


📄 시트: 1.현재흡연


,구분,Unnamed: 3,Unnamed: 4,2008,2009,2010,2011,2012,2013,2014,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,95% CI
0,서울,N,(명),"21,913","23,175","22,897","23,115","23,059","23,136","23,026",...,"22,950","22,955","22,904","22,928","22,908","22,920","22,943","22,928","22,903",NaN
1,NaN,전체,(조율),24.2(0.3),24.3(0.3),23.2(0.3),22.3(0.3),22.2(0.3),21.7(0.3),20.6(0.3),...,19.5(0.3),18.8(0.3),18.6(0.3),16.7(0.3),15.5(0.3),14.9(0.3),15.3(0.3),16.6(0.3),14.8(0.3),14.2-15.3
2,NaN,NaN,(표준화율),24.6(0.3),24.7(0.3),23.7(0.3),23.0(0.3),23.0(0.3),22.6(0.3),21.4(0.3),...,20.5(0.3),20.0(0.3),19.8(0.3),17.8(0.3),16.6(0.3),15.8(0.3),16.3(0.3),17.9(0.3),15.7(0.3),15.2-16.3
3,NaN,남,NaN,45.6(0.6),45.5(0.5),43.5(0.5),42.0(0.5),41.6(0.5),40.6(0.5),38.5(0.5),...,36.6(0.5),35.1(0.5),34.4(0.5),30.9(0.5),28.9(0.5),27.9(0.5),28.1(0.5),30.1(0.5),26.6(0.5),25.7-27.5
4,NaN,여,NaN,3.7(0.2),4.0(0.2),3.6(0.2),3.5(0.2),3.7(0.2),3.7(0.2),3.4(0.2),...,3.4(0.2),3.3(0.2),3.6(0.2),3.5(0.2),3.0(0.2),2.7(0.2),3.5(0.2),4.0(0.2),3.9(0.2),3.5-4.3
5,NaN,19-29,NaN,24.3(0.8),24.9(0.7),23.8(0.7),22.5(0.8),22.6(0.7),20.7(0.7),20.3(0.7),...,18.7(0.7),19.6(0.7),18.9(0.7),16.8(0.7),16.3(0.7),15.6(0.7),16.3(0.7),18.1(0.7),15.1(0.7),13.7-16.5
6,NaN,30-39,NaN,30.8(0.7),30.8(0.7),29.8(0.7),28.7(0.7),29.1(0.7),28.3(0.7),24.4(0.7),...,24.0(0.7),22.8(0.7),22.2(0.7),17.9(0.7),16.4(0.7),14.3(0.6),16.4(0.7),16.7(0.7),14.7(0.7),13.4-16.0
7,NaN,40-49,NaN,26.8(0.7),27.0(0.7),26.3(0.7),26.7(0.6),26.8(0.7),26.9(0.7),25.8(0.7),...,24.2(0.7),24.1(0.7),23.3(0.7),22.1(0.7),19.6(0.7),18.8(0.7),17.3(0.7),21.0(0.7),17.9(0.7),16.5-19.2
8,NaN,50-59,NaN,21.6(0.7),22.4(0.7),20.7(0.7),19.6(0.7),19.1(0.6),20.5(0.7),20.7(0.7),...,20.3(0.7),19.1(0.6),20.0(0.7),19.0(0.7),18.4(0.7),18.2(0.6),19.9(0.7),20.6(0.7),18.9(0.6),17.7-20.2
9,NaN,60-69,NaN,16.5(0.7),15.3(0.7),13.9(0.7),14.5(0.7),13.8(0.7),12.5(0.6),13.9(0.7),...,14.2(0.6),12.0(0.6),13.7(0.6),12.9(0.6),12.6(0.6),12.6(0.6),12.3(0.6),14.0(0.6),13.3(0.5),12.2-14.3



📄 시트: 7.월간음주


,구분,Unnamed: 3,Unnamed: 4,2008,2009,2010,2011,2012,2013,2014,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,95% CI
0,서울,N,(명),"21,880","23,178","22,877","23,115","23,057","23,137","23,025",...,"22,939","22,949","22,900","22,925","22,907","22,919","22,943","22,927","22,905",NaN
1,NaN,전체,(조율),57.7(0.4),57.8(0.4),59.8(0.4),61.4(0.4),59.1(0.4),60.7(0.4),60.3(0.4),...,61.5(0.4),61.0(0.4),59.7(0.4),58.7(0.4),53.0(0.4),50.9(0.4),55.3(0.4),56.3(0.4),56.2(0.4),55.4-56.9
2,NaN,NaN,(표준화율),57.8(0.3),58.1(0.3),60.4(0.3),62.3(0.3),60.3(0.3),62.3(0.3),62.2(0.3),...,63.8(0.3),63.8(0.3),62.8(0.3),62.1(0.4),56.5(0.4),54.9(0.4),59.4(0.4),60.7(0.4),60.5(0.4),59.8-61.2
3,NaN,남,NaN,75.3(0.5),75.7(0.5),76.4(0.5),77.6(0.5),76.0(0.4),77.1(0.5),75.8(0.4),...,76.8(0.5),74.8(0.5),73.8(0.5),71.9(0.5),66.5(0.5),63.9(0.5),67.9(0.5),67.7(0.5),67.7(0.5),66.7-68.6
4,NaN,여,NaN,40.8(0.5),40.5(0.5),43.9(0.5),45.9(0.5),42.9(0.5),45.1(0.5),45.6(0.5),...,46.9(0.5),47.9(0.5),46.3(0.5),46.3(0.5),40.3(0.5),38.7(0.5),43.6(0.5),45.7(0.5),45.6(0.5),44.7-46.6
5,NaN,19-29,NaN,66.6(0.9),66.7(0.8),68.7(0.8),71.3(0.8),68.7(0.8),71.9(0.8),71.3(0.8),...,73.6(0.8),72.5(0.8),71.0(0.8),71.0(0.8),63.2(0.9),58.6(0.9),68.2(0.8),67.9(0.9),65.8(0.9),64.0-67.6
6,NaN,30-39,NaN,66.5(0.7),66.3(0.7),67.7(0.7),69.1(0.7),67.7(0.7),68.9(0.7),69.1(0.7),...,71.6(0.8),70.6(0.7),69.0(0.8),68.0(0.9),64.1(0.9),63.3(0.9),66.4(0.9),69.0(0.9),67.9(0.9),66.2-69.7
7,NaN,40-49,NaN,61.6(0.8),62.4(0.7),64.8(0.8),67.2(0.7),66.0(0.7),66.6(0.7),66.8(0.7),...,66.4(0.8),69.7(0.8),68.4(0.8),67.5(0.8),60.8(0.9),61.6(0.9),62.0(0.9),65.2(0.9),65.7(0.9),64.0-67.5
8,NaN,50-59,NaN,51.4(0.9),52.4(0.9),55.9(0.8),58.1(0.8),55.0(0.8),58.3(0.8),58.8(0.8),...,61.3(0.8),59.7(0.8),58.9(0.8),58.6(0.9),54.6(0.9),52.3(0.8),57.7(0.8),57.6(0.9),58.4(0.9),56.7-60.1
9,NaN,60-69,NaN,40.6(1.0),39.7(0.9),44.6(1.0),44.6(1.0),42.0(0.9),43.4(1.0),44.0(0.9),...,46.1(0.9),45.2(0.9),46.4(0.9),45.2(0.9),41.0(0.8),37.9(0.9),42.9(0.8),43.3(0.8),45.0(0.8),43.5-46.5



📄 시트: 13.비만율


,구분,Unnamed: 3,Unnamed: 4,2008,2009,2010,2011,2012,2013,2014,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,95% CI
0,서울,N,(명),"21,586","23,152","22,639","22,822","22,611","22,827","22,660",...,"22,590","22,647","22,293",NaN,"22,628","22,751","22,763","22,856","22,816",NaN
1,NaN,전체,(조율),20.6(0.3),21.4(0.3),21.7(0.3),22.7(0.3),23.7(0.3),23.2(0.3),23.9(0.3),...,25.7(0.3),25.7(0.3),28.8(0.3),NaN,28.2(0.3),28.6(0.3),29.5(0.3),28.2(0.3),30.1(0.3),29.5-30.8
2,NaN,NaN,(표준화율),20.7(0.3),21.4(0.3),21.7(0.3),22.5(0.3),23.4(0.3),23.1(0.3),23.6(0.3),...,25.5(0.3),25.5(0.3),28.6(0.3),NaN,28.5(0.3),29.2(0.3),30.2(0.4),28.8(0.3),31.0(0.4),30.3-31.7
3,NaN,남,NaN,26.6(0.5),27.7(0.5),28.5(0.5),29.8(0.5),31.0(0.5),30.6(0.5),31.7(0.5),...,34.1(0.5),34.5(0.5),37.6(0.5),NaN,38.1(0.5),38.5(0.5),39.7(0.5),38.9(0.5),40.0(0.5),39.0-41.0
4,NaN,여,NaN,14.7(0.4),15.3(0.4),15.1(0.3),15.9(0.4),16.5(0.4),16.0(0.3),16.2(0.3),...,17.6(0.4),17.3(0.4),20.4(0.4),NaN,18.9(0.4),19.4(0.4),20.0(0.4),18.4(0.4),21.0(0.4),20.3-21.8
5,NaN,19-29,NaN,12.8(0.6),13.9(0.6),13.1(0.6),15.4(0.6),15.1(0.7),14.6(0.7),14.4(0.6),...,17.4(0.7),18.9(0.7),21.0(0.8),NaN,22.8(0.7),23.5(0.8),24.2(0.9),23.0(0.8),25.9(0.9),24.2-27.6
6,NaN,30-39,NaN,20.1(0.6),22.0(0.6),22.0(0.6),23.5(0.7),24.5(0.6),24.5(0.6),24.0(0.7),...,26.8(0.7),25.1(0.7),28.2(0.8),NaN,29.7(0.9),30.6(0.8),33.7(0.8),30.7(0.8),34.3(0.9),32.6-36.0
7,NaN,40-49,NaN,24.1(0.7),23.0(0.7),23.4(0.7),23.6(0.6),25.7(0.7),26.1(0.7),27.5(0.7),...,28.3(0.7),28.8(0.7),32.2(0.8),NaN,31.6(0.8),32.2(0.8),32.2(0.9),31.1(0.8),32.7(0.9),31.0-34.3
8,NaN,50-59,NaN,24.3(0.7),24.9(0.8),24.8(0.8),25.8(0.7),26.3(0.7),23.6(0.7),25.6(0.7),...,26.9(0.7),27.7(0.7),30.8(0.8),NaN,29.2(0.8),29.4(0.7),29.9(0.7),29.2(0.8),30.6(0.8),29.0-32.1
9,NaN,60-69,NaN,25.6(0.9),26.2(0.9),27.9(0.9),27.5(0.9),29.3(0.9),28.5(0.9),28.1(0.9),...,27.8(0.8),29.6(0.8),31.0(0.8),NaN,27.6(0.7),29.2(0.8),29.1(0.8),28.4(0.7),29.4(0.7),28.0-30.8



📄 시트: 17.스트레스


,구분,Unnamed: 3,Unnamed: 4,2008,2009,2010,2011,2012,2013,2014,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,95% CI
0,서울,N,(명),"21,911","23,172","22,866","23,104","23,047","23,136","23,023",...,"22,943","22,943","22,902","22,919","22,910","22,919","22,940","22,928","22,906",NaN
1,NaN,전체,(조율),30.4(0.4),31.1(0.4),29.5(0.3),30.6(0.3),30.6(0.3),28.7(0.3),29.7(0.3),...,29.1(0.3),26.8(0.3),26.4(0.3),25.8(0.3),27.8(0.3),26.3(0.3),24.0(0.3),25.4(0.3),24.0(0.3),23.3-24.6
2,NaN,NaN,(표준화율),30.6(0.3),31.2(0.3),29.8(0.3),31.0(0.3),31.1(0.3),29.4(0.3),30.6(0.3),...,30.1(0.3),28.1(0.3),27.7(0.3),27.2(0.3),29.5(0.4),28.0(0.3),25.7(0.3),27.1(0.4),25.9(0.4),25.2-26.6
3,NaN,남,NaN,31.6(0.5),32.0(0.5),30.1(0.5),30.7(0.5),30.1(0.5),28.4(0.5),30.0(0.5),...,29.3(0.5),25.8(0.5),26.0(0.5),23.9(0.5),26.3(0.5),24.5(0.4),22.7(0.5),24.2(0.5),22.8(0.4),22.0-23.7
4,NaN,여,NaN,29.3(0.5),30.3(0.5),29.0(0.4),30.6(0.4),31.0(0.4),29.0(0.4),29.5(0.4),...,28.9(0.4),27.7(0.4),26.8(0.4),27.6(0.5),29.2(0.4),27.9(0.4),25.2(0.4),26.6(0.4),25.0(0.4),24.2-25.9
5,NaN,19-29,NaN,27.9(0.7),30.5(0.8),30.3(0.8),32.5(0.8),33.9(0.8),30.7(0.8),32.3(0.8),...,30.9(0.8),30.5(0.8),29.6(0.8),30.1(0.9),30.0(0.8),29.6(0.8),26.6(0.8),28.2(0.9),26.4(0.9),24.7-28.2
6,NaN,30-39,NaN,34.4(0.7),36.6(0.7),34.3(0.7),33.6(0.7),35.2(0.8),34.4(0.7),35.9(0.8),...,36.0(0.8),34.3(0.8),32.7(0.8),32.0(0.8),35.5(0.9),33.2(0.8),29.8(0.9),31.3(0.8),31.2(0.8),29.6-32.8
7,NaN,40-49,NaN,34.1(0.8),32.9(0.8),32.3(0.7),33.6(0.8),32.7(0.7),31.3(0.7),33.3(0.7),...,32.3(0.7),30.5(0.7),29.9(0.8),28.8(0.8),33.3(0.8),30.7(0.8),29.2(0.8),29.5(0.8),28.7(0.8),27.0-30.3
8,NaN,50-59,NaN,28.5(0.8),30.4(0.8),26.7(0.8),27.1(0.8),27.5(0.8),25.3(0.7),26.0(0.7),...,27.0(0.8),23.5(0.7),23.8(0.7),22.6(0.7),26.7(0.7),25.0(0.7),22.5(0.7),25.9(0.8),23.2(0.7),21.8-24.7
9,NaN,60-69,NaN,26.9(0.9),24.2(0.9),21.9(0.8),25.1(0.8),22.1(0.9),21.2(0.8),20.8(0.8),...,19.8(0.8),17.3(0.7),18.3(0.7),19.6(0.7),20.3(0.7),19.3(0.7),17.0(0.7),18.3(0.6),17.5(0.6),16.3-18.8



📄 시트: 18.우울감


,구분,Unnamed: 3,Unnamed: 4,2008,2009,2010,2011,2012,2013,2014,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,95% CI
0,서울,N,(명),"21,916","23,171","22,871","23,109","23,055","23,135","23,024",...,"22,933","22,945","22,903","22,909","22,912","22,923","22,941","22,928","22,904",NaN
1,NaN,전체,(조율),8.2(0.2),7.9(0.2),6.8(0.2),7.0(0.2),6.8(0.2),7.1(0.2),8.3(0.2),...,7.0(0.2),7.2(0.2),5.1(0.2),6.5(0.2),6.3(0.2),7.0(0.2),7.3(0.2),8.4(0.2),7.5(0.2),7.1-7.9
2,NaN,NaN,(표준화율),8.1(0.2),7.8(0.2),6.7(0.2),6.9(0.2),6.7(0.2),7.0(0.2),8.1(0.2),...,6.9(0.2),7.1(0.2),4.9(0.2),6.5(0.2),6.3(0.2),6.8(0.2),7.1(0.2),8.3(0.2),7.3(0.2),6.9-7.8
3,NaN,남,NaN,5.3(0.2),5.3(0.2),4.6(0.2),4.9(0.2),4.9(0.2),5.0(0.2),6.2(0.3),...,5.4(0.2),5.6(0.2),3.6(0.2),5.0(0.2),4.8(0.2),5.2(0.2),5.3(0.2),6.3(0.3),6.0(0.3),5.5-6.5
4,NaN,여,NaN,10.9(0.3),10.3(0.3),9.0(0.3),9.1(0.3),8.7(0.3),9.1(0.3),10.3(0.3),...,8.4(0.3),8.7(0.3),6.4(0.2),8.0(0.3),7.7(0.3),8.6(0.3),9.1(0.3),10.2(0.3),8.8(0.3),8.3-9.4
5,NaN,19-29,NaN,6.2(0.4),6.6(0.4),6.1(0.4),6.9(0.5),7.5(0.5),6.2(0.4),7.2(0.4),...,7.7(0.5),7.9(0.5),4.7(0.4),7.4(0.5),7.3(0.5),7.1(0.5),8.1(0.5),8.8(0.5),7.9(0.5),6.9-9.0
6,NaN,30-39,NaN,7.2(0.4),6.4(0.4),5.6(0.3),5.6(0.4),5.4(0.3),6.5(0.4),7.6(0.4),...,6.4(0.4),6.4(0.4),4.6(0.4),6.1(0.4),6.8(0.5),5.7(0.4),7.0(0.4),8.0(0.5),7.2(0.5),6.3-8.1
7,NaN,40-49,NaN,8.0(0.4),7.2(0.4),6.7(0.4),6.3(0.4),5.7(0.4),6.8(0.4),7.3(0.4),...,6.0(0.4),6.6(0.4),4.8(0.4),5.7(0.4),5.1(0.4),6.7(0.4),6.1(0.4),7.9(0.5),6.5(0.5),5.6-7.4
8,NaN,50-59,NaN,9.3(0.6),9.4(0.5),7.9(0.5),7.2(0.4),7.6(0.4),6.8(0.4),8.8(0.5),...,6.5(0.4),7.2(0.4),4.8(0.4),6.4(0.4),6.5(0.4),7.5(0.4),6.5(0.4),8.6(0.5),7.4(0.4),6.6-8.3
9,NaN,60-69,NaN,10.7(0.7),10.3(0.6),7.3(0.5),8.9(0.6),8.0(0.5),9.1(0.6),10.1(0.6),...,7.6(0.5),7.1(0.5),5.6(0.4),6.5(0.4),6.1(0.4),7.0(0.4),8.1(0.5),7.7(0.4),7.5(0.4),6.7-8.3


In [ ]:
import re

def load_original_range(xls, sheet_name):
    df = xls.parse(sheet_name, skiprows=11, usecols="C:W", nrows=188)
    df.columns = df.columns.astype(str)
    df['region'] = df['구분'].ffill()
    df['group'] = df['Unnamed: 3'].astype(str).str.strip()
    df['unit'] = df['Unnamed: 4'].astype(str).str.strip()
    return df

def extract_values(df, indicator_name):
    year_cols = [str(y) for y in range(2008, 2025)]
    rows = []
    for _, row in df.iterrows():
        if row['group'] == '전체' and row['unit'] == '(조율)':
            for year in year_cols:
                val = row[year]
                if isinstance(val, str):
                    match = re.match(r"([\d\.]+)\(([\d\.]+)\)", val)
                    if match:
                        rows.append({
                            'region': row['region'],
                            'year': int(year),
                            'value': float(match.group(1)),
                            'std': float(match.group(2)),
                            'indicator': indicator_name
                        })
    return pd.DataFrame(rows)

In [ ]:
sheet_mapping = {
    '흡연율': '1.현재흡연',
    '음주율': '7.월간음주',
    '비만율': '13.비만율',
    '우울감': '18.우울감',
    '스트레스': '17.스트레스'
}

all_rows = []
for name, sheet in sheet_mapping.items():
    df_sheet = load_original_range(xls_province, sheet)
    df_extracted = extract_values(df_sheet, name)
    all_rows.append(df_extracted)

df_predict_ready = pd.concat(all_rows, ignore_index=True)

In [ ]:
# 지표별로 그룹 나눠서 샘플 출력
for indicator_name in df_predict_ready['indicator'].unique():
    print(f"\n지표: {indicator_name}")
    display(df_predict_ready[df_predict_ready['indicator'] == indicator_name].head())



지표: 흡연율


,region,year,value,std,indicator
0,서울,2008,24.2,0.3,흡연율
1,서울,2009,24.3,0.3,흡연율
2,서울,2010,23.2,0.3,흡연율
3,서울,2011,22.3,0.3,흡연율
4,서울,2012,22.2,0.3,흡연율



지표: 음주율


,region,year,value,std,indicator
285,서울,2008,57.7,0.4,음주율
286,서울,2009,57.8,0.4,음주율
287,서울,2010,59.8,0.4,음주율
288,서울,2011,61.4,0.4,음주율
289,서울,2012,59.1,0.4,음주율



지표: 비만율


,region,year,value,std,indicator
570,서울,2008,20.6,0.3,비만율
571,서울,2009,21.4,0.3,비만율
572,서울,2010,21.7,0.3,비만율
573,서울,2011,22.7,0.3,비만율
574,서울,2012,23.7,0.3,비만율



지표: 우울감


,region,year,value,std,indicator
838,서울,2008,8.2,0.2,우울감
839,서울,2009,7.9,0.2,우울감
840,서울,2010,6.8,0.2,우울감
841,서울,2011,7.0,0.2,우울감
842,서울,2012,6.8,0.2,우울감



지표: 스트레스


,region,year,value,std,indicator
1123,서울,2008,30.4,0.4,스트레스
1124,서울,2009,31.1,0.4,스트레스
1125,서울,2010,29.5,0.3,스트레스
1126,서울,2011,30.6,0.3,스트레스
1127,서울,2012,30.6,0.3,스트레스
